In [1]:
import tensorflow as tf
!pip install pandas
!pip install tensorflow_hub
!apt install -y git
# !git clone https://github.com/LydiaXiaohongLi/Albert_Finetune_with_Pretrain_on_Custom_Corpus.git
# !git clone https://github.com/kwonmha/bert-vocab-builder
# !git clone https://github.com/google-research/ALBERT
!pip install sentencepiece
!pip install six==1.12.0
#!git clone https://github.com/brightmart/albert_zh.git

     |████████████████████████████████| 10.1MB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 512kB 53.6MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 92kB 3.6MB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  git-man less libbsd0 libcurl3-gnutls libedit2 liberror-perl libssl1.0.0
  libx11-6 libx11-data libxau6 libxcb1 libxdmcp6 libxext6 libxmuu1
  openssh-client xauth
Suggested packages:
  gettext-base git-daemon-run | git-daemon-sysvinit git-doc git-el git-email
  git-gui gitk gitweb git-cvs git-mediawiki git-svn keychain libpam-ssh
  monkeysphere ssh-askpass
The following NEW packages will be installed:
  git git-man less libbsd0 libcurl3-gnutls libedit2 liberror-perl libssl1.0.0
  libx11-6 libx11

7Progress: [ 23%] [#############.............................................] 87Progress: [ 24%] [##############............................................] 8Selecting previously unselected package less.
Preparing to unpack .../07-less_487-0.1_amd64.deb ...
7Progress: [ 26%] [##############............................................] 8Unpacking less (487-0.1) ...
7Progress: [ 27%] [###############...........................................] 87Progress: [ 28%] [################..........................................] 8Selecting previously unselected package libedit2:amd64.
Preparing to unpack .../08-libedit2_3.1-20170329-1_amd64.deb ...
7Progress: [ 29%] [################..........................................] 8Unpacking libedit2:amd64 (3.1-20170329-1) ...
7Progress: [ 30%] [#################.........................................] 87Progress: [ 31%] [##################........................................] 8Selecting previously unselected package libssl1.

7Progress: [ 93%] [#####################################################.....] 87Progress: [ 94%] [######################################################....] 8Setting up libxext6:amd64 (2:1.3.3-1) ...
7Progress: [ 95%] [#######################################################...] 87Progress: [ 97%] [#######################################################...] 8Setting up xauth (1:1.0.10-1) ...
7Progress: [ 98%] [########################################################..] 87Progress: [ 99%] [#########################################################.] 8Processing triggers for mime-support (3.60ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...

78Collecting sentencepiece
     |████████████████████████████████| 1.0MB 3.2MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:

import pandas as pd
import pickle

In [7]:
import numpy as np
a=np.array([-1,2,4,6])
a=a +4000
a[-1]

4006

In [3]:
with open('cmc.data.final.pickle', 'rb') as data:
    dataset = pickle.load(data)

X_train=dataset['train'][0]
y_train=dataset['train'][1]

X_test=dataset['test'][0]
y_test=dataset['test'][1]

In [4]:
import re
sample=X_train['cmp_str']
dmn = r'[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
url= r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
ip= r'(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}'
mc= r'[a-zA-Z]{5}[0-9][a-zA-Z]{1,4}[0-9]{1,4}'
stop_words=['Success','INTERNAL','ALERT']
perc=r'(\d\%)'
mlls=r'(\d*\.)?\d+'
intgr_end=r'\s\d+$'
intgr_val=r'\s\d+\s'
intgr_val_start=r'^\d+\s'
number_decimal=r'\s(\d*\.)?\d+\s'
num=r'[0-9]+'

sample_modified=[]
for sentence in sample:
    sentence=re.sub(dmn, ' ##dom ', sentence)   
    sentence=re.sub(url, ' ##url ', sentence)   
    sentence=re.sub(ip, ' ##ip ', sentence)   
    sentence=re.sub(mc, ' ##mc ', sentence)   
    #sentence=re.sub("[\u0000-\u007F]+", ' ', sentence)
    #sentence=re.sub("[\x00-\x7F]+", ' ', sentence)
    
    sentence=re.sub(perc, ' ', sentence)
    sentence=re.sub(mlls, ' ', sentence)
    sentence=re.sub(number_decimal, ' ', sentence)
    sentence=re.sub(intgr_val, ' ', sentence)
    sentence=re.sub(intgr_val_start, ' ', sentence)
    sentence=re.sub(intgr_end, ' ', sentence)
    sentence=re.sub('\d+', ' * ', sentence)
    sentence=re.sub('\"', ' ', sentence)
    sentence=re.sub('\\\\\\\\\\\\', ' ', sentence)
    sentence=re.sub('\\\\', ' ', sentence)
    sentence=re.sub('[\\-/()}{\":,;.\[\]]+', ' ', sentence)
    # sentence=re.sub('\"\"[0-9a-zA-Z]+\s', ' ', sentence)
    # sentence=re.sub('\"\"[a-zA-Z]+\s', ' ', sentence)
    # sentence=re.sub('[\s]+', ' ', sentence)
    #sentence=re.sub('[\\]', ' ', sentence)
    for stp in stop_words:
        sentence= sentence.replace(stp,'')
    sample_modified.append(sentence)

In [20]:
!mkdir data
X_train['cmp_str_new']=sample_modified
print(X_train.shape)
X_train['cmp_str_new'].to_csv("data/training.data.full.csv",index=False,header=False)
X_train['cmp_str_new'][:20000].to_csv("data/training.data.20000.csv",index=False,header=False)

mkdir: cannot create directory ‘data’: File exists
(510766, 21)


In [9]:
!cd /home/ubuntu/abasu/docker/bert-vocab-builder/ && python subword_builder.py  \
!  --corpus_filepattern /home/ubuntu/abasu/docker/data/training.data.full.csv  \
!--output_filename /home/ubuntu/abasu/docker/data/vocab.txt --min_count 1



W0215 09:27:48.207941 139675017987840 module_wrapper.py:139] From /home/ubuntu/abasu/docker/bert-vocab-builder/tokenizer.py:133: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

['/home/ubuntu/abasu/docker/data/training.data.full.csv']

W0215 09:27:48.209454 139675017987840 module_wrapper.py:139] From /home/ubuntu/abasu/docker/bert-vocab-builder/tokenizer.py:138: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.

read 100000 lines, 3.0521535873413086 secs elapsed
read 200000 lines, 5.754852533340454 secs elapsed
read 300000 lines, 8.49962568283081 secs elapsed
read 400000 lines, 11.32930612564087 secs elapsed
read 500000 lines, 14.414829969406128 secs elapsed
14.739990711212158 for reading read file : /home/ubuntu/abasu/docker/data/training.data.full.csv
read all files

W0215 09:28:02.956886 139675017987840 module_wrapper.py:139] From /home/ubuntu/abasu/docker/bert-vocab-builder/text_encoder.py:588: The name tf.logging.info is depr

In [21]:
!cd  /home/ubuntu/abasu/docker/ALBERT/ && python create_pretraining_data.py \
! --input_file /home/ubuntu/abasu/docker/data/training.data.20000.csv \
! --output_file  /home/ubuntu/abasu/docker/data/training.data.tfrecord.20000.txt  \
! --vocab_file  /home/ubuntu/abasu/docker/data/vocab.txt  --max_seq_length=64

INFO:tensorflow:*** Reading from input files ***
I0215 09:58:52.837670 140221396559616 create_pretraining_data.py:631] *** Reading from input files ***
INFO:tensorflow:  /home/ubuntu/abasu/docker/data/training.data.20000.csv
I0215 09:58:52.837974 140221396559616 create_pretraining_data.py:633]   /home/ubuntu/abasu/docker/data/training.data.20000.csv
INFO:tensorflow:number of instances: 361176
I0215 10:07:14.939020 140221396559616 create_pretraining_data.py:641] number of instances: 361176
INFO:tensorflow:*** Writing to output files ***
I0215 10:07:14.939912 140221396559616 create_pretraining_data.py:644] *** Writing to output files ***
INFO:tensorflow:  /home/ubuntu/abasu/docker/data/training.data.tfrecord.20000.txt
I0215 10:07:14.940014 140221396559616 create_pretraining_data.py:646]   /home/ubuntu/abasu/docker/data/training.data.tfrecord.20000.txt
INFO:tensorflow:*** Example ***
I0215 10:07:14.940903 140221396559616 create_pretraining_data.py:188] *** Example ***
INFO:tensorflow:toke

I0215 10:07:14.967759 140221396559616 create_pretraining_data.py:200] input_ids: 3 2990 4830 2780 4879 4861 216 4868 4879 2780 4871 4877 4869 4875 4860 384 1999 122 4879 4862 4876 1999 2990 4861 4868 4879 4860 4871 4877 4869 4875 4860 4861 127 124 1507 1507 4 5 430 56 56 1659 5 2364 5 5 176 122 4879 4862 4876 5 5 75 47 546 5 427 4861 4854 4876 4860 4
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
I0215 10:07:14.967885 140221396559616 create_pretraining_data.py:200] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
I0215 10:07:14.968003 140221396559616 create_pretraining_data.py:200] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
!cd /home/ubuntu/abasu/docker/albert_zh/ &&  python run_pretraining_google.py \
! --input_file=/home/ubuntu/abasu/docker//data/training.data.tfrecord.20000.txt \
! --output_dir=/home/ubuntu/abasu/docker/models/ \
! --albert_config_file=/home/ubuntu/abasu/docker/data/albert_config.json  \
! --train_batch_size=32  --num_train_steps=10000 --max_seq_length=64 




W0215 10:20:31.325236 139857143273216 module_wrapper.py:139] From run_pretraining_google.py:472: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0215 10:20:31.325464 139857143273216 module_wrapper.py:139] From run_pretraining_google.py:472: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0215 10:20:31.325662 139857143273216 module_wrapper.py:139] From /home/ubuntu/abasu/docker/albert_zh/modeling_google.py:112: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0215 10:20:31.326420 139857143273216 module_wrapper.py:139] From run_pretraining_google.py:479: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0215 10:20:31.326634 139857143273216 module_wrapper.py:139] From run_pretraining_google.py:483: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0215 10:20:31.327955 139857143273216 module_wra

Instructions for updating:
Use `tf.data.experimental.map_and_batch()
W0215 10:20:32.316826 139857143273216 deprecation.py:323] From run_pretraining_google.py:449: map_and_batch_with_legacy_function (from tensorflow.python.data.experimental.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.map_and_batch()

W0215 10:20:32.390650 139857143273216 module_wrapper.py:139] From /home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/autograph/converters/directives.py:119: The name tf.parse_single_example is deprecated. Please use tf.io.parse_single_example instead.

Instructions for updating:
Use `tf.cast` instead.
W0215 10:20:32.517679 139857143273216 deprecation.py:323] From run_pretraining_google.py:465: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:<DatasetV1Adapter shapes: {input_

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0215 10:20:33.635939 139857143273216 deprecation.py:323] From /home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1375: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
I0215 10:20:35.879310 139857143273216 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
I0215 10:20:35.880987 139857143273216 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
I0215 10:20:36.850476 139857143273216 monitored_session.py:240] Graph was finalized.
2020-02-15 10:20:36.851003: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2

INFO:tensorflow:global_step/sec: 8.17313
I0215 10:20:45.732382 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.17313
INFO:tensorflow:examples/sec: 261.54
I0215 10:20:45.732841 139857143273216 tpu_estimator.py:2308] examples/sec: 261.54
INFO:tensorflow:global_step/sec: 8.31942
I0215 10:20:45.852587 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.31942
INFO:tensorflow:examples/sec: 266.222
I0215 10:20:45.853032 139857143273216 tpu_estimator.py:2308] examples/sec: 266.222
INFO:tensorflow:global_step/sec: 8.12133
I0215 10:20:45.975713 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.12133
INFO:tensorflow:examples/sec: 259.883
I0215 10:20:45.976153 139857143273216 tpu_estimator.py:2308] examples/sec: 259.883
INFO:tensorflow:global_step/sec: 8.14216
I0215 10:20:46.098532 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.14216
INFO:tensorflow:examples/sec: 260.549
I0215 10:20:46.098972 139857143273216 tpu_estimator.py:2308] examples/sec: 260.549
INFO:tenso

INFO:tensorflow:global_step/sec: 8.14063
I0215 10:20:49.828640 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.14063
INFO:tensorflow:examples/sec: 260.5
I0215 10:20:49.829061 139857143273216 tpu_estimator.py:2308] examples/sec: 260.5
INFO:tensorflow:global_step/sec: 8.09441
I0215 10:20:49.952184 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.09441
INFO:tensorflow:examples/sec: 259.021
I0215 10:20:49.952622 139857143273216 tpu_estimator.py:2308] examples/sec: 259.021
INFO:tensorflow:global_step/sec: 8.32131
I0215 10:20:50.072355 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.32131
INFO:tensorflow:examples/sec: 266.282
I0215 10:20:50.072793 139857143273216 tpu_estimator.py:2308] examples/sec: 266.282
INFO:tensorflow:global_step/sec: 8.38229
I0215 10:20:50.191642 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.38229
INFO:tensorflow:examples/sec: 268.233
I0215 10:20:50.192096 139857143273216 tpu_estimator.py:2308] examples/sec: 268.233
INFO:tensorf

...

INFO:tensorflow:global_step/sec: 8.13502
I0215 10:27:35.138370 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.13502
INFO:tensorflow:examples/sec: 260.321
I0215 10:27:35.138833 139857143273216 tpu_estimator.py:2308] examples/sec: 260.321
INFO:tensorflow:global_step/sec: 8.14899
I0215 10:27:35.261115 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.14899
INFO:tensorflow:examples/sec: 260.768
I0215 10:27:35.261548 139857143273216 tpu_estimator.py:2308] examples/sec: 260.768
INFO:tensorflow:global_step/sec: 8.27315
I0215 10:27:35.381953 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.27315
INFO:tensorflow:examples/sec: 264.741
I0215 10:27:35.382406 139857143273216 tpu_estimator.py:2308] examples/sec: 264.741
INFO:tensorflow:global_step/sec: 8.33627
I0215 10:27:35.501928 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.33627
INFO:tensorflow:examples/sec: 266.761
I0215 10:27:35.502415 139857143273216 tpu_estimator.py:2308] examples/sec: 266.761
INFO:ten

INFO:tensorflow:global_step/sec: 8.2552
I0215 10:27:39.286084 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.2552
INFO:tensorflow:examples/sec: 264.167
I0215 10:27:39.286517 139857143273216 tpu_estimator.py:2308] examples/sec: 264.167
INFO:tensorflow:global_step/sec: 8.26821
I0215 10:27:39.407011 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.26821
INFO:tensorflow:examples/sec: 264.583
I0215 10:27:39.407471 139857143273216 tpu_estimator.py:2308] examples/sec: 264.583
INFO:tensorflow:global_step/sec: 8.0457
I0215 10:27:39.531290 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.0457
INFO:tensorflow:examples/sec: 257.462
I0215 10:27:39.531741 139857143273216 tpu_estimator.py:2308] examples/sec: 257.462
INFO:tensorflow:global_step/sec: 8.27746
I0215 10:27:39.652105 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.27746
INFO:tensorflow:examples/sec: 264.879
I0215 10:27:39.652567 139857143273216 tpu_estimator.py:2308] examples/sec: 264.879
INFO:tensorf

INFO:tensorflow:global_step/sec: 8.12027
I0215 10:27:43.443685 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.12027
INFO:tensorflow:examples/sec: 259.849
I0215 10:27:43.444137 139857143273216 tpu_estimator.py:2308] examples/sec: 259.849
INFO:tensorflow:global_step/sec: 8.32291
I0215 10:27:43.563812 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.32291
INFO:tensorflow:examples/sec: 266.333
I0215 10:27:43.564265 139857143273216 tpu_estimator.py:2308] examples/sec: 266.333
INFO:tensorflow:global_step/sec: 8.22472
I0215 10:27:43.685397 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.22472
INFO:tensorflow:examples/sec: 263.191
I0215 10:27:43.685842 139857143273216 tpu_estimator.py:2308] examples/sec: 263.191
INFO:tensorflow:global_step/sec: 8.2769
I0215 10:27:43.806211 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.2769
INFO:tensorflow:examples/sec: 264.861
I0215 10:27:43.806650 139857143273216 tpu_estimator.py:2308] examples/sec: 264.861
INFO:tenso

INFO:tensorflow:global_step/sec: 8.28135
I0215 10:27:47.594979 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.28135
INFO:tensorflow:examples/sec: 265.003
I0215 10:27:47.595427 139857143273216 tpu_estimator.py:2308] examples/sec: 265.003
INFO:tensorflow:global_step/sec: 8.32661
I0215 10:27:47.715080 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.32661
INFO:tensorflow:examples/sec: 266.451
I0215 10:27:47.715580 139857143273216 tpu_estimator.py:2308] examples/sec: 266.451
INFO:tensorflow:global_step/sec: 8.13315
I0215 10:27:47.838039 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.13315
INFO:tensorflow:examples/sec: 260.261
I0215 10:27:47.838468 139857143273216 tpu_estimator.py:2308] examples/sec: 260.261
INFO:tensorflow:global_step/sec: 8.09739
I0215 10:27:47.961551 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.09739
INFO:tensorflow:examples/sec: 259.117
I0215 10:27:47.962000 139857143273216 tpu_estimator.py:2308] examples/sec: 259.117
INFO:ten

INFO:tensorflow:global_step/sec: 8.01782
I0215 10:27:51.757831 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.01782
INFO:tensorflow:examples/sec: 256.57
I0215 10:27:51.758286 139857143273216 tpu_estimator.py:2308] examples/sec: 256.57
INFO:tensorflow:global_step/sec: 8.08346
I0215 10:27:51.881539 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.08346
INFO:tensorflow:examples/sec: 258.671
I0215 10:27:51.881991 139857143273216 tpu_estimator.py:2308] examples/sec: 258.671
INFO:tensorflow:global_step/sec: 8.29065
I0215 10:27:52.002153 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.29065
INFO:tensorflow:examples/sec: 265.301
I0215 10:27:52.002631 139857143273216 tpu_estimator.py:2308] examples/sec: 265.301
INFO:tensorflow:global_step/sec: 8.09002
I0215 10:27:52.125770 139857143273216 tpu_estimator.py:2307] global_step/sec: 8.09002
INFO:tensorflow:examples/sec: 258.881
I0215 10:27:52.126209 139857143273216 tpu_estimator.py:2308] examples/sec: 258.881
INFO:tenso

In [17]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [16]:
!rm -r models

In [27]:
10*(20000/32)

6250.0